In [1]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [2]:
import tensorflow_datasets as tfds
print(tfds.__version__)

import pathlib

1.3.2


In [12]:
import matplotlib
%matplotlib inline

# Load & Prepare data

In [22]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True, 
    as_supervised=True, # Return (image, label) instead of ({"image": image, "label": label})
    with_info=True,
)

In [23]:
info.features

FeaturesDict({
    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})

## Build training pipeline

TFDS provides image as tf.uint8, we need tf.float32. Therefore, normalize images

In [25]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

In [26]:
ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

MNIST dataset is small so we can cache to RAM. This should be call before any random transformation

In [27]:
ds_train = ds_train.cache()

Shuffle, Batch, Prefetch

In [28]:
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

## Build evaluation pipeline

No shuffle here

In [29]:
ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

# Create model

In [37]:
from tensorflow.keras import layers

In [38]:
# model = tf.keras.models.Sequential([
#     layers.Conv2D(input_shape=(28, 28, 1)),
#     layers.Dense(128,activation='relu'),
#     layers.Dense(10, activation='softmax')
# ])

In [61]:
model = tf.keras.models.Sequential([
    layers.Conv2D(32, 3, input_shape=(28, 28, 1), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPool2D(),
    layers.Dropout(0.5),
    layers.Conv2D(32, 3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [62]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_7 (Batch (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_8 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 10, 10, 32)      

In [63]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

# Train

In [64]:
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 5s 11ms/step - loss: 0.1425 - accuracy: 0.9559 - val_loss: 0.5755 - val_accuracy: 0.8156
Epoch 2/6
469/469 [==============================] - 4s 8ms/step - loss: 0.0481 - accuracy: 0.9849 - val_loss: 0.0399 - val_accuracy: 0.9870
Epoch 3/6
469/469 [==============================] - 4s 8ms/step - loss: 0.0356 - accuracy: 0.9888 - val_loss: 0.0450 - val_accuracy: 0.9857
Epoch 4/6
469/469 [==============================] - 4s 8ms/step - loss: 0.0287 - accuracy: 0.9910 - val_loss: 0.0421 - val_accuracy: 0.9885
Epoch 5/6
469/469 [==============================] - 4s 8ms/step - loss: 0.0219 - accuracy: 0.9927 - val_loss: 0.0303 - val_accuracy: 0.9909
Epoch 6/6
469/469 [==============================] - 4s 8ms/step - loss: 0.0209 - accuracy: 0.9934 - val_loss: 0.0376 - val_accuracy: 0.9895
